In [1]:
import pandas as pd
import os
import numpy as np
import numpy.ma as ma
import glob
from netCDF4 import Dataset,date2num,num2date
import numpy as np
import pandas as pd
import warnings
import sympy as sp
import math
warnings.filterwarnings('ignore')
import xlsxwriter

In [2]:
def extract(Var,min_index_lat,min_index_lon):
    cycle=np.array([Var[min_index_lat,min_index_lon],Var[min_index_lat+1,min_index_lon],
                    Var[min_index_lat-1,min_index_lon],Var[min_index_lat,min_index_lon+1],
                    Var[min_index_lat,min_index_lon-1]])#,Var[min_index_lat+1,min_index_lon+1],Var[min_index_lat+1,min_index_lon-1],
                    #Var[min_index_lat-1,min_index_lon+1],Var[min_index_lat-1,min_index_lon-1]])
    cycle[cycle==999]=np.nan
    a=np.nanmean(cycle)
    return a
def QPE_three(X1,X2,X3,a,b,c,d):
    rr_data = a*np.power(X1,b)*np.power(X2,c)*np.power(X3,d)
    return rr_data
def QPE_dual(X1,X2,a, b,c):
    rr_data = a*np.power(X1,b)*np.power(X2,c)
    return rr_data
def QPE_sing(X1,a,b):
    rr_data = a*np.power(X1,b)
    return rr_data

In [3]:
#read informations of gauge stations
stations=pd.read_excel(pd.ExcelFile("C:/Users/Admin/OneDrive/Desktop/gauges2.xlsx"),sheet_name='3')
#Choose a Case: Embedded/Squall line/Full stratiform
#Testing-Full stratiform:16,17,18,19,20/Squall line:16,17,18,19/Embedded:16,17,18,19,20,21,22
Case='General.xlsx'
#Choose events of a case
Case_dic={Case:[str(i) for i in[*range(28,51,1)]]}
frames=[]
for r in Case_dic[Case]:
    #make gauges dataframe at each events
    df_gauges=pd.read_excel('E:/data/Gauges/Finall/'+Case,sheet_name=r)
    #make dataframes for each equal
    linkNC='D:/data/SVM_netCDF/'+Case[:-5]+'/'+r+'/'#'E:/SVM_netCDF/'+Case[:-5]+'/'+r+'/'
    df_CS_SVM=pd.DataFrame(0,columns=df_gauges.columns[3:],index=stations['Tên Trạm'])
    df_R_Z_ZDR_KDP=pd.DataFrame(0,columns=df_gauges.columns[3:],index=stations['Tên Trạm'])
    df_R_Z_ZDR=pd.DataFrame(0,columns=df_gauges.columns[3:],index=stations['Tên Trạm'])
    df_R_Z=pd.DataFrame(0,columns=df_gauges.columns[3:],index=stations['Tên Trạm'])
    df_R_KDP=pd.DataFrame(0,columns=df_gauges.columns[3:],index=stations['Tên Trạm'])
    df_R_Z_noSVM=pd.DataFrame(0,columns=df_gauges.columns[3:],index=stations['Tên Trạm'])
    df_R_Marshall=pd.DataFrame(0,columns=df_gauges.columns[3:],index=stations['Tên Trạm'])
    df_event=pd.DataFrame(0,columns=df_gauges.columns[3:],index=stations['Tên Trạm'])
    n=0
    for file_name in os.listdir(linkNC):
        data=Dataset(linkNC+file_name,'r') # at 1 time
        CS_SVM=data.variables['CS_SVM'][0,:,:]
        Zh=np.ma.power(10.,0.1*data.variables['Zh_1.5km'][0,:,:])
        ZDR=np.ma.power(10.,0.1*data.variables['ZDR_1.5km'][0,:,:])
        KDP=data.variables['KDP_1.5km'][0,:,:]
        #R(Z,ZDR,KDP)
        #General_S(a=2.11,b=0.20,c=-0.86,d=0.19),General_C(a=64.50,b=-0.03,c=-2.25,d=0.50)
        R_CS_Z_ZDR_KDP=data.variables['CS_SVM'][:][0,:,:]
        R_CS_Z_ZDR_KDP[CS_SVM==1]=QPE_three(Zh[CS_SVM==1],ZDR[CS_SVM==1],KDP[CS_SVM==1],a=2.11,b=0.20,c=-0.86,d=0.19)
        R_CS_Z_ZDR_KDP[CS_SVM==2]=QPE_three(Zh[CS_SVM==2],ZDR[CS_SVM==2],KDP[CS_SVM==2],a=64.50,b=-0.03,c=-2.25,d=0.50)
        R_CS_Z_ZDR_KDP.filled(fill_value=999)
        #R(Z,ZDR)
        #General_S(a=1.19,b=0.23,c=-0.77),General_C(a=3.03,b=0.28,c=-2.10)
        R_CS_Z_ZDR=data.variables['CS_SVM'][:][0,:,:]
        R_CS_Z_ZDR[CS_SVM==1]=QPE_dual(Zh[CS_SVM==1],ZDR[CS_SVM==1],a=1.19,b=0.23,c=-0.77)
        R_CS_Z_ZDR[CS_SVM==2]=QPE_dual(Zh[CS_SVM==2],ZDR[CS_SVM==2],a=3.03,b=0.28,c=-2.10)
        R_CS_Z_ZDR.filled(fill_value=999)
        #R(Z)
        #General_S(a=1.15,b=0.21),General_C(a=3.05,b=0.19)
        R_CS_Z=data.variables['CS_SVM'][:][0,:,:]
        R_CS_Z[CS_SVM==1]=QPE_sing(Zh[CS_SVM==1],a=1.15,b=0.21)
        R_CS_Z[CS_SVM==2]=QPE_sing(Zh[CS_SVM==2],a=3.05,b=0.19)
        R_CS_Z.filled(fill_value=999)
        #R(KDP)
        #General_S(a=7.34,b=0.38),General_C(a=21.86,b=0.47)
        R_CS_KDP=data.variables['CS_SVM'][:][0,:,:]
        R_CS_KDP[CS_SVM==1]=QPE_sing(KDP[CS_SVM==1],a=7.34,b=0.38)
        R_CS_KDP[CS_SVM==2]=QPE_sing(KDP[CS_SVM==2],a=21.86,b=0.47)
        R_CS_KDP.filled(fill_value=999)
        #R(Z)- no SVM
        #a=1.02,b=0.24
        R_Z_noSVM=QPE_sing(Zh,a=1.02,b=0.24)
        R_Z_noSVM.filled(fill_value=999)
        #Rm(Z)-Marshall
        R_Marshall=QPE_sing(Zh,a=0.0365,b=0.625)
        R_Marshall.filled(fill_value=999)
        for index,row in stations.iterrows():
            locations=row['Tên Trạm']
            location_lat=row['Latitude']
            location_lon=row['Longitude']
            lat=data.variables['latitude'][:]
            lon=data.variables['longitude'][:]
            sq_diff_lat=(lat-location_lat)**2
            sq_diff_lon=(lon-location_lon)**2
            min_index_lat=sq_diff_lat.argmin()
            min_index_lon=sq_diff_lon.argmin()
            df_CS_SVM.loc[[locations],list(df_gauges.columns[3:])[n]]=round(extract(CS_SVM,min_index_lat,min_index_lon),0)
            df_R_Z_ZDR_KDP.loc[[locations],list(df_gauges.columns[3:])[n]]=extract(R_CS_Z_ZDR_KDP,min_index_lat,min_index_lon)
            df_R_Z_ZDR.loc[[locations],list(df_gauges.columns[3:])[n]]=extract(R_CS_Z_ZDR,min_index_lat,min_index_lon)
            df_R_Z.loc[[locations],list(df_gauges.columns[3:])[n]]=extract(R_CS_Z,min_index_lat,min_index_lon)
            df_R_KDP.loc[[locations],list(df_gauges.columns[3:])[n]]=extract(R_CS_KDP,min_index_lat,min_index_lon)
            df_R_Marshall.loc[[locations],list(df_gauges.columns[3:])[n]]=extract(R_Marshall,min_index_lat,min_index_lon)
            df_R_Z_noSVM.loc[[locations],list(df_gauges.columns[3:])[n]]=extract(R_Z_noSVM,min_index_lat,min_index_lon)
            df_event.loc[[locations],list(df_gauges.columns[3:])[n]]=(int(r)-27)
        n=n+1
    df=pd.DataFrame(data={
    'Ranges': list(df_gauges['Ranges'][:])*df_gauges.columns[3:].size,
    'Stations': list(df_gauges['Trạm Đo'][:])*df_gauges.columns[3:].size,
    'times': list(np.repeat(df_gauges.columns[3:],df_gauges['Trạm Đo'].size)),
    'Event':list(pd.Series(df_event.iloc[:,:].values.ravel('F'))),
    'SC_SVM':list(pd.Series(df_CS_SVM.iloc[:,:].values.ravel('F'))),
    'R_10p':list(pd.Series(df_gauges.iloc[:,3:].values.ravel('F'))*6),
    'R_Z_ZDR_KDP':list(pd.Series(df_R_Z_ZDR_KDP.iloc[:,:].values.ravel('F'))),   
    'R_Z_ZDR':list(pd.Series(df_R_Z_ZDR.iloc[:,:].values.ravel('F'))),
    'R_Z':list(pd.Series(df_R_Z.iloc[:,:].values.ravel('F'))),
    'R_KDP':list(pd.Series(df_R_KDP.iloc[:,:].values.ravel('F'))),
    'R_Z_noSVM':list(pd.Series(df_R_Z_noSVM.iloc[:,:].values.ravel('F'))),
    'R_Marshall':list(pd.Series(df_R_Marshall.iloc[:,:].values.ravel('F')))
    #'R_CS_SJ':list(pd.Series(df_R_CS_SJ.iloc[:,:].values.ravel('F')))
    #'Z_1500m':list(pd.Series(df_Zh.iloc[:,:].values.ravel('F'))),
    #'ZDR_1500m':list(pd.Series(df_ZDR.iloc[:,:].values.ravel('F')))  
    })
    frames.append(df)
    print(r)
result = pd.concat(frames,ignore_index=True)
#df_result_Z_ZDR_KDP=result.loc[(result.R_10p>0)&(result.R_Z_ZDR_KDP>0)].drop(['R_Z_ZDR','R_Z','R_Marshall','R_Z_noSVM'], axis=1)
#df_result_Z_ZDR=result.loc[(result.R_10p>0)&(result.R_Z_ZDR>0)].drop(['R_Z_ZDR_KDP','R_KDP'], axis=1)
#zz=[df_result_Z_ZDR_KDP,df_result_Z_ZDR]
#sheets=['R_Z_ZDR_KDP','R_Z_ZDR']
#out_path = r"E:/data/evaluation_QPE_PA6/"+Case
#writer = pd.ExcelWriter(out_path,engine='xlsxwriter')   
#for dataframe, sheet in zip(zz, sheets):
#    dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0,index=False)   
#writer.save()

28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [4]:
result

,Ranges,Stations,times,Event,SC_SVM,R_10p,R_Z_ZDR_KDP,R_Z_ZDR,R_Z,R_KDP,R_Z_noSVM,R_Marshall
0,15.22,Mường Sại,05:10 19/02,1,1.0,0.0,3.045635,3.337996,3.540314,2.815965,3.694638,1.113705
1,17.72,Ta Ma,05:10 19/02,1,1.0,0.0,1.384585,1.657349,1.889421,1.975101,1.801981,0.169050
2,19.16,Chiềng Ơn,05:10 19/02,1,1.0,1.2,1.918143,1.821746,2.105414,2.923943,2.038259,0.229891
3,22.09,Thuận Châu,05:10 19/02,1,1.0,8.4,3.846708,3.913477,4.377266,3.523290,4.703681,2.017319
4,23.07,Nậm Ét,05:10 19/02,1,1.0,9.6,2.622229,2.207408,2.509883,4.379157,2.504557,0.466547
...,...,...,...,...,...,...,...,...,...,...,...,...
33985,137.89,Mường Bang 1,13:00 26/08,23,1.0,0.0,NaN,1.231990,1.212238,NaN,1.083328,0.042699
33986,139.88,Bảo Ái 1,13:00 26/08,23,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
33987,146.26,Mường Khương,13:00 26/08,23,1.0,1.2,NaN,1.326950,1.378471,NaN,1.254874,0.062897
33988,147.45,Vĩnh Lạc,13:00 26/08,23,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
#df_finall=result.loc[((result.R_10p>0)&(result.SC_SVM==1.0)) | ((result.R_10p>0)&(result.SC_SVM==2.0)) ]

In [6]:
df_finall=result.loc[(result.R_10p>0)]
df_finall

,Ranges,Stations,times,Event,SC_SVM,R_10p,R_Z_ZDR_KDP,R_Z_ZDR,R_Z,R_KDP,R_Z_noSVM,R_Marshall
2,19.16,Chiềng Ơn,05:10 19/02,1,1.0,1.2,1.938125,1.847162,2.116123,2.998409,2.057208,0.229891
3,22.09,Thuận Châu,05:10 19/02,1,1.0,8.4,3.984554,4.066718,4.249103,3.630758,4.585162,2.017319
4,23.07,Nậm Ét,05:10 19/02,1,1.0,9.6,2.691390,2.259225,2.498031,4.539060,2.502082,0.466547
7,31.40,Nậm Giôn,05:10 19/02,1,1.0,40.8,6.307759,4.585667,6.169790,9.170340,4.939534,3.142856
8,35.36,Chiềng Khay,05:10 19/02,1,1.0,1.2,NaN,1.079178,1.401919,NaN,1.281282,0.063666
...,...,...,...,...,...,...,...,...,...,...,...,...
22853,109.30,Mường Hum,13:00 26/08,23,1.0,1.2,2.818325,3.513201,3.528368,2.299308,3.701140,1.109181
22855,112.25,Cốc Mỳ 1,13:00 26/08,23,1.0,1.2,1.503077,1.693049,1.806689,2.324821,1.714464,0.138148
22856,114.29,Mường Vi,13:00 26/08,23,1.0,7.2,1.957698,2.335186,2.421847,2.142152,2.401090,0.342535
22857,114.99,Bản Lang,13:00 26/08,23,1.0,1.2,2.774177,3.006713,2.904411,2.803091,2.957974,0.595977


In [7]:
df_finall.to_excel('detec_event_PA6.xlsx')